<h1 style="text-align: center;"> Losers Queue Analysis </h1>

<p>This notebook contains code used extract match information from Riot's API and analyse it, with the goal of better understanding the League of Legends Matchmaking system and if there are any evidence of the existence of a losers queue</p>


In [ ]:
from riotwatcher import LolWatcher, ApiError
# import pandas as pd #for some reason pypy isnt able to install pandas so im not using it for now
import json
import time

# stored in a different file so I dont accidentally upload my
riot_api_key = open("riot_api_key.txt", "r")
# api key to git
watcher = LolWatcher(riot_api_key.read())
my_region = 'euw1'
queue = 'RANKED_SOLO_5x5'


def downloadChallengerIds(region, queue):
    '''
        Downloads all 300 challenger account ids from the desired queue and returns a list containing all
        of those.
    '''
    list_of_challengers = watcher.league.challenger_by_queue(my_region, queue)
    challenger_accountIds = []
    for challenger in list_of_challengers['entries']:
        summoner = watcher.summoner.by_id(
            my_region, challenger['summonerId'])   # we do this step because
        # the challenger_by_queue method does not return AccountId information. In order to get the accountId
        # we need to call the summoner.by_id() method and get the accountID returned.
        # I can only make a limited amount of requests per 2 minutes, so im using this to not get
        time.sleep(1)
        # an error
        challenger_accountIds.append(summoner['accountId'])
    return challenger_accountIds

# challenger_accountIds = downloadChallengerIds(my_region, queue)
# with open('challenger_account_ids.json', 'w') as f:  # saves the accountIds to disk
#     json.dump(challenger_accountIds, f)


# with open('challenger_account_ids.json') as f:  # reads the chall account ids saved previously
#     challenger_accountIds = json.load(f)


def downloadMatchesById(region, accountId, queue):
    '''
        Returns a list with all last 100 matches played by the accountId
    '''
    detailed_matches_list = []
    matches = watcher.match.matchlist_by_account(
        region, accountId, queue)  # gets all the 100 matches ids
    for match in matches['matches']:
        # waits for a sec so we don't surpass the max amount of requests per minute
        time.sleep(1)
        #start = time.time()
        match_detail = watcher.match.by_id(region, match['gameId'])
        #print(f'It took {time.time() - start} seconds to request the match')
        #start = time.time()
        # this will make filtering easier in mongoDB later
        match_detail["_id"] = match_detail["gameId"]
        detailed_matches_list.append(match_detail)
#         print(
#             f'It took {time.time() - start} seconds to append the match to the list')
    return detailed_matches_list


def downloadMatchesChallenger(region, queue):
    '''
        Returns a list with all the unique matches played by the challenger players.
    '''
    challenger_accountIds = downloadChallengerIds(region, queue)
    gameIds = []  # gameId is the information we need to request detailed match info
    detailed_matches_list = []
    for challengerId in challenger_accountIds[0:2]:
        time.sleep(1)
        matches = watcher.match.matchlist_by_account(
            region, challengerId, queue)  # gets all the 100 matches ids
        for match in matches['matches']:
            gameIds.append(match['gameId'])
    print(len(gameIds))  # should be 30k long   
    gameIds = list(set(gameIds))  # removes duplicates
    print(len(gameIds))  # should only count unique gameids
    
    try:
        for gameId in gameIds:
            time.sleep(10)
            match_detail = watcher.match.by_id(region, gameId)
            # this will make filtering easier in mongoDB later
            match_detail["_id"] = match_detail["gameId"]
            detailed_matches_list.append(match_detail)
        return detailed_matches_list
    except:
        print('probably requested too much information at once')
    finally:
        return(detailed_matches_list)


challenger_matches = downloadMatchesChallenger(my_region, queue)


In [ ]:
len(challenger_matches)
# print(challenger_matches)

In [ ]:
# Creating a MongoDB to store match information
import pymongo

dbclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = dbclient["losers_queue"]
players = mydb["players"]
matches = mydb['matches']

In [ ]:
# for each challenger ID, gets information about 100 matches and put them inside mongoDB 'matches'
# collection
matches.insert_many(challenger_matches, ordered=False)

<p> Now that we have added all the match information from the 300 top ranked challenger players, we need to analyze those.  </p>

In [ ]:
# Extracting matches from the collection

summoner = "Lorde Poeira"
query = { "participantIdentities.player.summonerName:" f"{summoner}"}
for match in matches.find(query):
  print(match)


    
# def lastMatch():
#     '''
#         Asks for a summoner name and tells you if that summoner has won its last match or not
#     '''
#     summonerName = input('Type your summoner name: ').casefold()
#     try:
#         summoner = watcher.summoner.by_name(my_region, summonerName)
#     except ApiError as err:
#         if err.response.status_code == 429:
#             print('this retry-after is handled by default by the RiotWatcher library')
#             print('future requests wait until the retry-after time passes')
#         elif err.response.status_code == 404:
#             print('Summoner with that name not found.')
#             raise
#         else:
#             raise

#     # all objects are returned (by default) as a dict

#     # my_ranked_stats = watcher.league.by_summoner(my_region, summoner['id'])
#     # # print(my_ranked_stats)

#     my_matches = watcher.match.matchlist_by_account(my_region, summoner['accountId'])


#     # fetch last match detail
#     last_match = my_matches['matches'][0]
#     match_detail = watcher.match.by_id(my_region, last_match['gameId'])

#     participants = []
#     for row in range(0, 10):
#         participants_row = {'win': match_detail['participants'][row]['stats']['win'],
#                             'SummonerName': match_detail['participantIdentities'][row]['player']['summonerName'].casefold(),
#                             }
#         participants.append(participants_row)


#     df = pd.DataFrame(participants)

#     if not df['win'][df['SummonerName'] == summonerName].empty:
#         if df['win'][df['SummonerName'] == summonerName].bool():
#             print('You won your last match.')
#         else:
#             print('You lost your last match')
#     else:
#         print('Couldn\'t find information about your last match')
